In [1]:
	
print("hello")

hello


In [2]:
import os
os.listdir()


['apple.csv', 'laptop_price.csv', 'sample.ipynb']

In [3]:
import pandas as pd
import matplotlib as pt

data=pd.read_csv("laptop_price.csv",encoding="latin1")
# print(data.head())
# print(data) #this is just the hardware specs and in general for any country by comparison
print(data.columns)

#Step-1: we loaded it the data we had to give that encding as it's a dataset from spain and so we encode it
#now we see the columns in the dataset. 
# IMP******************
# NOTE: this object'data' is created in my laptop's RAM and any changes temporary to that only fed to model

Index(['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros'],
      dtype='object')


In [4]:
#Step-2: In this step since we cant feed words into model so we convert them to numbers** 
#the units like GB and all should be changed RAM is mostly commonly used for price comparison we start with it.

data["Ram_GB"]=data["Ram"].str.replace("GB","").astype(int) # and we store it to a new column be doing so
print(data[["Ram","Ram_GB"]])

       Ram  Ram_GB
0      8GB       8
1      8GB       8
2      8GB       8
3     16GB      16
4      8GB       8
...    ...     ...
1298   4GB       4
1299  16GB      16
1300   2GB       2
1301   6GB       6
1302   4GB       4

[1303 rows x 2 columns]


In [5]:
#Step-2: Same thing as RAM we apply to GPU for same reasons. Numbers are fed to the model not words
# this is not as simple as RAM so we use a helper function to check the presence required GPU brands and then store in that column for each row of GPU 
# we use it.

def get_gpu_brand(gpu):
    gpu= gpu.lower() #as some will be upper and lower case mixed so we do this
    if "nvidia" in gpu:
        return "nvidia"
    elif "amd" in gpu or "radeon" in gpu:
        return "amd"
    elif "intel" in gpu:
        return "intel"
    else:
        return "other"
    
data["Gpu_Brand"]=data["Gpu"].apply(get_gpu_brand)
print(data[["Gpu","Gpu_Brand"]].head(10))
    

                            Gpu Gpu_Brand
0  Intel Iris Plus Graphics 640     intel
1        Intel HD Graphics 6000     intel
2         Intel HD Graphics 620     intel
3            AMD Radeon Pro 455       amd
4  Intel Iris Plus Graphics 650     intel
5                 AMD Radeon R5       amd
6       Intel Iris Pro Graphics     intel
7        Intel HD Graphics 6000     intel
8          Nvidia GeForce MX150    nvidia
9        Intel UHD Graphics 620     intel


In [6]:
#Step-3: similar to the GPU we use helper function to extract the brand name for CPU

def extract_cpu_brand(cpu):
    cpu=cpu.lower()
    if "intel" in cpu:
        return "intel"
    elif "amd" in cpu:
        return "amd"
    else:
        return "other"
    
data["Cpu_brand"]=data["Cpu"].apply(extract_cpu_brand)
print(data[["Cpu","Cpu_brand"]].head(15))

                           Cpu Cpu_brand
0         Intel Core i5 2.3GHz     intel
1         Intel Core i5 1.8GHz     intel
2   Intel Core i5 7200U 2.5GHz     intel
3         Intel Core i7 2.7GHz     intel
4         Intel Core i5 3.1GHz     intel
5      AMD A9-Series 9420 3GHz       amd
6         Intel Core i7 2.2GHz     intel
7         Intel Core i5 1.8GHz     intel
8   Intel Core i7 8550U 1.8GHz     intel
9   Intel Core i5 8250U 1.6GHz     intel
10  Intel Core i5 7200U 2.5GHz     intel
11    Intel Core i3 6006U 2GHz     intel
12        Intel Core i7 2.8GHz     intel
13    Intel Core i3 6006U 2GHz     intel
14      Intel Core M m3 1.2GHz     intel


In [7]:
features=[
    "Ram_GB",
    "Weight",
    "Inches"
]

In [8]:
feature_cols=[col for col in data.columns if col.startswith(("Gpu_Brand","Cpu_Brand", "Company", "TypeName","OpSys"))]
# what is this thing

In [9]:
# One-hot encode the categorical features
X = pd.get_dummies(data[features + ["Gpu_Brand", "Cpu_brand"]], drop_first=True)

y=data["Price_euros"] #this is the labelled output/ Targets

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, random_state=42)

In [11]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
# print(data.head())

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [12]:
from sklearn.metrics import mean_absolute_error
preds=model.predict(X_test) #the predicted values after training on the hypothesis
print("MAE:",mean_absolute_error(y_test, preds))

MAE: 265.51512293462525


In [13]:
# now the testing part
sample=X.iloc[0].copy()
sample["Ram_GB"] = 32
sample["GPU_Brand_nvidia"] = 1
sample["GPU_Brand_intel"] = 0
sample["GPU_Brand_amd"] = 0
sample["Cpu_brand_intel"] = 0
sample["Cpu_brand_amd"] = 1


sample_df = pd.DataFrame([sample])
# Align sample_df with X_train columns
sample_df = sample_df.reindex(columns=X_train.columns, fill_value=False)
predicted_eur=model.predict(sample_df)[0]

def euro_to_inr(predicted_eur):
    eur_to_inr = 90
    predicted_eur = model.predict(sample_df)[0]
    predicted_inr = predicted_eur * eur_to_inr
    return predicted_inr
    
print("Predicted price:",euro_to_inr(predicted_eur) )


Predicted price: 318732.91348198545
